VEP Analysis Code
AJK
02-17-2024

In [28]:
# Load packages and data

import mne
import numpy as np
from mne.preprocessing import (ICA)
#from autoreject import AutoReject
import matplotlib
matplotlib.use("TkAgg")

eeg_path = "Adam_VEP"  # You will need to change the file name

file_eeg = eeg_path + ".eeg"
file_vhdr = eeg_path + ".vhdr"
file_vmrk = eeg_path + ".vmrk"

raw = mne.io.read_raw_brainvision(file_vhdr)
#drop_channels = ['BIP2','EOG','TEMP1','ACC1','ACC2','ACC3']
#raw = raw.drop_channels(drop_channels)
events_from_annot, event_dict = mne.events_from_annotations(raw)
del event_dict['Stimulus/s5']
#raw.set_channel_types({'BIP1':'ecg'})
raw.info

Extracting parameters from Adam_VEP.vhdr...
Setting channel info structure...
Used Annotations descriptions: ['Marker/Impedance', 'New Segment/', 'Stimulus/s1', 'Stimulus/s2', 'Stimulus/s3', 'Stimulus/s5', 'Stimulus/s9007', 'Stimulus/s9008']


C:\Users\neuro\AppData\Local\Temp\ipykernel_6864\1107138427.py:16: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_brainvision(file_vhdr)


Measurement date,"May 06, 2024 13:14:30 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,64 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,4000.00 Hz
Highpass,0.00 Hz
Lowpass,2000.00 Hz


In [29]:
# Filter data and create epochs

highpass = 0.5
lowpass = 40
notch = 60

raw_filtered = raw.load_data().filter(highpass, lowpass).notch_filter(np.arange(notch, (notch * 3), notch))
eeg_1020 = raw_filtered.copy().set_eeg_reference(ref_channels = 'average') #['Fz'])
ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
eeg_1020 = eeg_1020.set_montage(ten_twenty_montage, on_missing = 'ignore')
eeg_1020.info['bads'] = []
eeg_1020.plot()
picks = mne.pick_types(eeg_1020.info, meg=False, eeg=True, stim=False, eog=False, include=[], exclude=[])

epochs = mne.Epochs(eeg_1020,
                    events=events_from_annot,
                    event_id=event_dict,
                    tmin=-0.050,
                    tmax=0.300,   #duration of stimulus or response
                    baseline=None,
                    reject=None,
                    verbose=False,
                    preload=True,
                    detrend=None,
                    event_repeated='drop')

del raw, raw_filtered, ten_twenty_montage

Reading 0 ... 934717  =      0.000 ...   233.679 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 26401 samples (6.600 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 26401 samples (6.600 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


In [30]:
# Run AutoReject to find bad epochs

n_interpolates = np.array([1, 4, 32])
consensus_percs = np.linspace(0, 1.0, 11)
ar = AutoReject(n_interpolates,
                consensus_percs,
                picks=picks,
                thresh_method='random_search',
                random_state=42)  
epochs_ar = ar.fit_transform(epochs)
epochs_ar.plot()

NameError: name 'AutoReject' is not defined

In [ ]:
# Use ICA to remove cardiac artifacts

ica = ICA(n_components = 30, max_iter = 'auto', random_state = 123)
ica.fit(epochs_ar)
ica_z_thresh = 1.96
ica.exclude = []
epochs_clean = epochs_ar.copy()
ecg_indices, ecg_scores = ica.find_bads_ecg(epochs_clean,
                                            threshold=ica_z_thresh)
ica.exclude = ecg_indices
print(ecg_indices)
ica.apply(epochs_clean)
epochs_final = epochs_clean.copy()
epochs_final.plot()

del eeg_1020, epochs, epochs_clean

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 30 components
Fitting ICA took 31.5s.
[]
Applying ICA to Epochs instance
    Transforming to ICA space (30 components)
    Zeroing out 0 ICA components
    Projecting back using 63 PCA components
Opening epochs-browser...


In [19]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\neuro\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [31]:
# Manual ICA analysis (optional)

########################################
ica = ICA(n_components = 20, max_iter = 'auto', random_state = 123)
ica.fit(epochs)
#ica.plot_sources(epochs)
#ica.plot_components()
#ica.plot_properties(epochs)

exclude = [0, 1, 5]    # select based on ICA abnl. OPTIONAL.
epochs_clean = epochs.copy()
ica.exclude = exclude
ica.apply(epochs_clean)
epochs_clean.plot(n_channels = len(epochs_clean))
epochs_final = epochs_clean.copy()

## if no ICAs:
#epochs_final = epochs_clean.copy()
#del eeg_1020, epochs, epochs_clean

Fitting ICA to data using 64 channels (please be patient, this may take a while)
Selecting by number: 20 components
Fitting ICA took 1.7s.
Applying ICA to Epochs instance
    Transforming to ICA space (20 components)
    Zeroing out 3 ICA components
    Projecting back using 64 PCA components


In [32]:
# Average epochs by condition

VEP = epochs_final['Stimulus/s3'].apply_baseline().average()
blank = epochs_final['Stimulus/s2'].apply_baseline().average()

Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)


In [33]:
# Plot VEP
fig = mne.viz.plot_compare_evokeds(VEP, picks=['Oz'], show=True)
fig[0].savefig("VEP_Oz")

fig = mne.viz.plot_compare_evokeds(dict(Checkboard=VEP, Blank=blank), colors=dict(Checkboard="orange", Blank="black"), picks=['Oz', 'O1', 'O2'], combine="mean")
fig[0].savefig("VEP_Comparison_Occipital")

combining channels using "mean"
combining channels using "mean"


In [38]:
# Evaluate VEP properties

def print_peak_measures(ch, tmin, tmax, lat, amp):
    print(f"Channel: {ch}")
    print(f"Time Window: {tmin * 1e3:.3f} - {tmax * 1e3:.3f} ms")
    print(f"Peak Latency: {lat * 1e3:.3f} ms")
    print(f"Peak Amplitude: {amp * 1e6:.3f} µV")

VEP_ROI = VEP.copy().pick(['Oz','O1','O2'])

good_tmin, good_tmax = 0.05, 0.15
ch, lat, amp = VEP_ROI.get_peak(
    ch_type="eeg", tmin=good_tmin, tmax=good_tmax, mode="pos", return_amplitude=True
)
print("*** PEAK MEASURES ***")
print_peak_measures(ch, good_tmin, good_tmax, lat, amp)


good_tmin, good_tmax = 0.05, 0.15
ch, lat, amp = VEP_ROI.get_peak(
    ch_type="eeg", tmin=good_tmin, tmax=good_tmax, mode="neg", return_amplitude=True
)
print("*** TROUGH MEASURES ***")
print_peak_measures(ch, good_tmin, good_tmax, lat, amp)

*** PEAK MEASURES ***
Channel: O2
Time Window: 50.000 - 150.000 ms
Peak Latency: 130.000 ms
Peak Amplitude: 2.234 µV
*** TROUGH MEASURES ***
Channel: O1
Time Window: 50.000 - 150.000 ms
Peak Latency: 86.000 ms
Peak Amplitude: -2.070 µV
